<a href="https://colab.research.google.com/github/altaga/Pytorch-Driving-Guardian/blob/main/Hardware%2520Code/Jetson%2520Code/Drowsiness/Drowsiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
!wget https://github.com/altaga/Pytorch-Driving-Guardian/blob/main/Hardware%20Code/Jetson%20Code/Drowsiness/model/BlinkModel.t7?raw=true -O BlinkModel.t7
!wget https://i.ibb.co/YX91y16/c1.png
!wget https://i.ibb.co/7n8MZSL/c2.png
!wget https://i.ibb.co/tcPhrRC/c3.png
!wget https://raw.githubusercontent.com/altaga/Pytorch-Driving-Guardian/main/Hardware%20Code/Jetson%20Code/Drowsiness/utils/model.py
!wget https://raw.githubusercontent.com/altaga/Pytorch-Driving-Guardian/main/Hardware%20Code/Jetson%20Code/Drowsiness/utils/grad_cam.py

In [ ]:
import cv2
import matplotlib.pyplot as plt
from IPython import display
import model
import torch
import os
from torchvision import transforms
from PIL import Image
from grad_cam import BackPropagation

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 
MyModel="BlinkModel.t7"

In [ ]:
shape = (24,24)
classes = [
    'Close',
    'Open',
]

eyess=[]
cface=0

In [ ]:
def preprocess(image_path):
    global cface
    transform_test = transforms.Compose([
        transforms.ToTensor()
    ])
    image = cv2.imread(image_path['path'])    
    faces = face_cascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(1, 1),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    if len(faces) == 0:
        ...
    else:
        cface=1
        (x, y, w, h) = faces[0]
        face = image[y:y + h, x:x + w]
        cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,0),2)
        roi_color = image[y:y+h, x:x+w]
        """
        Depending on the quality of your camera, this number can vary 
        between 10 and 40, since this is the "sensitivity" to detect the eyes.
        """
        sensi=20
        eyes = eye_cascade.detectMultiScale(face,1.3, sensi) 
        i=0
        for (ex,ey,ew,eh) in eyes:
            (x, y, w, h) = eyes[i]
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            eye = face[y:y + h, x:x + w]
            eye = cv2.resize(eye, shape)
            eyess.append([transform_test(Image.fromarray(eye).convert('L')), eye, cv2.resize(face, (48,48))])
            i=i+1
    cv2.imwrite('display.jpg',image) 
    

def eye_status(image, name, net):
    img = torch.stack([image[name]])
    bp = BackPropagation(model=net)
    probs, ids = bp.forward(img)
    actual_status = ids[:, 0]
    prob = probs.data[:, 0]
    if actual_status == 0:
        prob = probs.data[:,1]

    #print(name,classes[actual_status.data], probs.data[:,0] * 100)
    return classes[actual_status.data]

def func(imag,modl):
    drow(images=[{'path': imag, 'eye': (0,0,0,0)}],model_name=modl)

def drow(images, model_name):
    global eyess
    global cface
    global timebasedrow
    global timebasedis
    global timerundrow
    global timerundis
    net = model.Model(num_classes=len(classes))
    checkpoint = torch.load(MyModel, map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['net'])
    net.eval()
    flag =1
    status=""
    for i, image in enumerate(images):
        if(flag):
            preprocess(image)
            flag=0
        if cface==0:
            image = cv2.imread("display.jpg")
            image = cv2.putText(image, 'No face Detected', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 0, 0), 10, cv2.LINE_AA)
            cv2.imwrite('display.jpg',image)
        elif(len(eyess)!=0):
            eye, eye_raw , face = eyess[i]
            image['eye'] = eye
            image['raw'] = eye_raw
            image['face'] = face
            for index, image in enumerate(images):
                status = eye_status(image, 'eye', net)
                if(status =="Close"):
                    image = cv2.imread("display.jpg")
                    image = cv2.putText(image, 'Distracted', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 0, 0), 10, cv2.LINE_AA)
                    cv2.imwrite('display.jpg',image)       
        else:
            image = cv2.imread("display.jpg")
            image = cv2.putText(image, 'Drowsy', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 7, (255, 0, 0), 10, cv2.LINE_AA)
            cv2.imwrite('display.jpg',image)

In [ ]:
eyess=[]
func("c1.png",MyModel)
image = cv2.imread("display.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
eyess=[]
func("c2.png",MyModel)
image = cv2.imread("display.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

In [ ]:
eyess=[]
func("c3.png",MyModel)
image = cv2.imread("display.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)